<a href="https://colab.research.google.com/github/amfolity/interpretability/blob/main/SoftmaxProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def adaptive_temperature_softmax(logits):
  original_probs = jax.nn.softmax(logits)
  poly_fit = jnp.array([-0.037, 0.481, -2.3, 4.917, -1.791]) # see Figure 5
  entropy = jnp.sum(-original_probs * jnp.log(original_probs + 1e-9), axis=-1, keepdims=True) # compute the Shannon entropy
  beta = jnp.where( # beta = 1 / theta
                   entropy > 0.5, # don’t overcorrect low-entropy heads
  jnp.maximum(jnp.polyval(poly_fit, entropy), 1.0), # never increase entropy
  1.0)
return jax.nn.softmax(logits * beta)

hi


In [ ]:
import transformers

In [ ]:
model_name = 'google/gemma-2-2b-it'


In [ ]:
from huggingface_hub import login
login()

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
model = transformers.AutoModelForCausalLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
for param in model.parameters():
  param.requires_grad = False

In [ ]:
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

In [ ]:
import torch
prompt = ''

In [ ]:
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False)
for i in range(10):
  next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
  batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
  batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

print(tokenizer.decode(batch['input_ids'][0]))

NameError: name 'tokenizer' is not defined

In [ ]:
batch

{'input_ids': tensor([[     2, 235280,   4320,   8426,  25341]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import jax.numpy as jnp
from flax import linen as nn
from typing import Callable
class Model(nn.Module):
  def __init__(self, n_classes, n_feats, activation):
    super().__init__()
  n_classes = n_classes # 10
  n_feats = n_feats # 128
  activation = nn.gelu


def __call__(self, x, q):
  x = nn.Dense(features=self.n_feats)(x)
  x = self.activation(x)
  x = nn.Dense(features=self.n_feats)(x)
  x = self.activation(x)
  q = nn.Dense(features=self.n_feats)(q)
  q = self.activation(q)
  q = nn.Dense(features=self.n_feats)(q)
  x = nn.MultiHeadDotProductAttention(
  num_heads=1,
  qkv_features=self.n_feats)(
  inputs_q=q,
  inputs_kv=x)
  x = nn.Dense(features=self.n_feats)(jnp.squeeze(x, -2))
  x = self.activation(x)
  x = nn.Dense(features=self.n_classes)(x)
  return x

In [69]:
import torch
class Model(nn.Module):
  def __init__(self, n_classes=10, n_feats=128, activation=nn.GELU()):
    super().__init__()
    self.n_classes = n_classes
    self.n_feats = n_feats
    self.activation = activation
    self.fc1 = nn.Linear(in_features=self.n_classes + 1, out_features=self.n_feats)
    self.fc2 = nn.Linear(in_features=self.n_feats, out_features=self.n_feats)
    self.fc3 = nn.Linear(in_features=self.n_feats, out_features=self.n_feats)
    self.fc4 = nn.Linear(in_features=self.n_feats, out_features=self.n_feats)
    self.q_proj = nn.Linear(in_features=self.n_feats, out_features=self.n_feats, bias=False)
    self.k_proj = nn.Linear(in_features=self.n_feats, out_features=self.n_feats, bias=False)
    self.v_proj = nn.Linear(in_features=self.n_feats, out_features=self.n_feats, bias=False)

    self.fc5 = nn.Linear(in_features=self.n_feats, out_features=self.n_feats)
    self.fc6 = nn.Linear(in_features=self.n_feats, out_features=self.n_classes)


  def forward(self, x, q):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    q = self.activation(self.fc3(q))
    q = self.fc4(q)
    k = self.k_proj(x)
    q = self.q_proj(q)
    v = self.v_proj(x)

    #print(q.shape)
    ##print(x.unsqueeze(-3).shape)
    #print(x.shape)

    x = F.scaled_dot_product_attention(q.unsqueeze(-3), k.unsqueeze(-3), v.unsqueeze(-3), is_causal=False)

    #print(x.shape)
    #print(x.squeeze(-2).squeeze(-2).shape)

    x = self.fc6(self.activation(self.fc5(x.squeeze(-2).squeeze(-2))))

    #print(x.shape)
    return x

In [70]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dtype = torch.float32 if device=='cuda' else torch.float32

In [71]:
batch_size = 128  # Number of samples in the batch
input_dim = 128   # Number of features per sample
n = 16
num_classes = 10
# Generate a batch of random values in the range [0,1)

rho = torch.rand(batch_size, n, 1)
cla = torch.randint(0, num_classes, (batch_size, n))
one_hot = torch.zeros(batch_size, n, num_classes)
one_hot.scatter_(2, cla.unsqueeze(2), 1)
X = torch.cat([rho, one_hot], dim=-1)
Y = cla.take(torch.argmax(rho.squeeze(-1), dim=-1))
q = torch.rand(batch_size, 1, input_dim)



#q =  torch.rand(1, input_dim)
model = Model(n_classes=num_classes, n_feats=input_dim, activation=nn.GELU())
#model(X,q)

In [72]:
model.to(device).to(dtype)

Model(
  (activation): GELU(approximate='none')
  (fc1): Linear(in_features=11, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=128, bias=True)
  (q_proj): Linear(in_features=128, out_features=128, bias=False)
  (k_proj): Linear(in_features=128, out_features=128, bias=False)
  (v_proj): Linear(in_features=128, out_features=128, bias=False)
  (fc5): Linear(in_features=128, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=10, bias=True)
)

In [73]:
model.fc3.weight

Parameter containing:
tensor([[-0.0052, -0.0009,  0.0453,  ..., -0.0737,  0.0836, -0.0221],
        [-0.0232,  0.0699,  0.0593,  ..., -0.0228, -0.0782,  0.0306],
        [-0.0524,  0.0332, -0.0398,  ...,  0.0253, -0.0750, -0.0840],
        ...,
        [-0.0211,  0.0541,  0.0544,  ...,  0.0661,  0.0051,  0.0125],
        [-0.0413,  0.0045,  0.0386,  ..., -0.0342, -0.0623,  0.0712],
        [ 0.0875,  0.0852,  0.0348,  ..., -0.0416, -0.0107, -0.0215]],
       device='cuda:0', requires_grad=True)

In [74]:
model.fc3.weight

Parameter containing:
tensor([[-0.0052, -0.0009,  0.0453,  ..., -0.0737,  0.0836, -0.0221],
        [-0.0232,  0.0699,  0.0593,  ..., -0.0228, -0.0782,  0.0306],
        [-0.0524,  0.0332, -0.0398,  ...,  0.0253, -0.0750, -0.0840],
        ...,
        [-0.0211,  0.0541,  0.0544,  ...,  0.0661,  0.0051,  0.0125],
        [-0.0413,  0.0045,  0.0386,  ..., -0.0342, -0.0623,  0.0712],
        [ 0.0875,  0.0852,  0.0348,  ..., -0.0416, -0.0107, -0.0215]],
       device='cuda:0', requires_grad=True)

In [75]:
torch.max(model.fc3.weight)

tensor(0.0884, device='cuda:0', grad_fn=<MaxBackward1>)

In [76]:
import random
import tqdm
opt = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.000)
#opt = torch.optim.Adam(model.parameters(), lr=0.01)
q = torch.rand(1, 1, input_dim, device=device, dtype=dtype).tile((batch_size, 1, 1))


for epoch in tqdm.trange(100000):
  n = random.randint(5, 16)
  rho = torch.rand(batch_size, n, 1, device=device, dtype=dtype)
  cla = torch.randint(0, num_classes, (batch_size, n), device=device)
  one_hot = torch.zeros(batch_size, n, num_classes, device=device, dtype=dtype)
  one_hot.scatter_(2, cla.unsqueeze(2), 1)
  X = torch.cat([rho, one_hot], dim=-1)
  Y = cla.take_along_dim(torch.argmax(rho.squeeze(-1), dim=-1).unsqueeze(-1), dim=-1).squeeze(-1)
  #q = torch.rand(batch_size, 1, input_dim, device=device, dtype=dtype)
  opt.zero_grad()
  outputs = model(X,q) ## what is X and q here, do we need to generate each tim
  loss = F.cross_entropy(outputs, Y)
  if epoch % 1000 == 0:
    print("loss is {}".format(loss.item()))
  loss.backward()
  opt.step()

  0%|          | 34/100000 [00:00<04:56, 337.71it/s]

loss is 2.3030314445495605


  1%|          | 1067/100000 [00:03<05:02, 327.23it/s]

loss is 0.07532951235771179


  2%|▏         | 2046/100000 [00:06<05:00, 325.71it/s]

...

loss is 0.0258495956659317


 12%|█▏        | 12041/100000 [00:37<04:18, 340.24it/s]

loss is 0.010263040661811829


 12%|█▏        | 12264/100000 [00:37<04:31, 322.81it/s]


KeyboardInterrupt: 

In [ ]:
q = torch.rand(batch_size, 1, input_dim, device=device).to(device)
X = X.to(device)

In [60]:
idx = 101
print(X[idx])
print(Y[idx])

tensor([[0.6077, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.0000, 0.0000],
        [0.4161, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 1.0000],
        [0.7344, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.8621, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.7682, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.7123, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.0000, 0.0000],
        [0.0200, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
         0.0000, 0.0000],
        [0.5742, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000]], device='cuda:0')
tensor(1, device='cuda:0')


In [26]:
print(rho[idx])
print(cla[idx])

tensor([[0.5275],
        [0.1077],
        [0.9811],
        [0.0509],
        [0.9447],
        [0.3840],
        [0.1458],
        [0.2467]], device='cuda:0')
tensor([0, 3, 0, 4, 4, 4, 1, 6], device='cuda:0')


In [29]:
rho.shape

torch.Size([128, 8, 1])

In [40]:
torch.argmax(rho.squeeze(-1), dim=-1).unsqueeze(-1)

tensor([[2],
        [2],
        [2],
        [2],
        [0],
        [0],
        [0],
        [4],
        [4],
        [5],
        [6],
        [2],
        [7],
        [0],
        [5],
        [3],
        [3],
        [3],
        [4],
        [4],
        [2],
        [5],
        [5],
        [4],
        [5],
        [4],
        [1],
        [1],
        [0],
        [4],
        [0],
        [5],
        [7],
        [7],
        [7],
        [6],
        [6],
        [5],
        [1],
        [5],
        [3],
        [4],
        [7],
        [5],
        [2],
        [0],
        [1],
        [4],
        [2],
        [3],
        [0],
        [3],
        [2],
        [5],
        [1],
        [2],
        [5],
        [6],
        [7],
        [7],
        [3],
        [0],
        [7],
        [4],
        [2],
        [6],
        [6],
        [5],
        [3],
        [3],
        [6],
        [1],
        [0],
        [5],
        [7],
        [0],
        [7],

In [46]:
cla.take_along_dim(torch.argmax(rho.squeeze(-1), dim=-1).unsqueeze(-1), dim=-1).squeeze(-1)

tensor([6, 8, 4, 0, 6, 1, 6, 9, 2, 7, 1, 3, 8, 0, 2, 0, 4, 6, 1, 7, 8, 0, 1, 2,
        3, 9, 8, 7, 0, 9, 4, 0, 0, 5, 3, 1, 5, 5, 6, 1, 9, 1, 2, 3, 9, 9, 5, 7,
        1, 8, 4, 2, 1, 9, 6, 4, 3, 8, 6, 2, 0, 8, 6, 1, 6, 5, 3, 4, 2, 7, 3, 2,
        0, 0, 6, 6, 4, 0, 4, 8, 6, 9, 3, 0, 8, 4, 5, 7, 9, 0, 2, 5, 3, 7, 7, 8,
        0, 3, 3, 5, 4, 9, 9, 8, 5, 9, 8, 4, 8, 7, 5, 3, 7, 4, 4, 9, 0, 1, 6, 4,
        4, 8, 0, 0, 8, 9, 0, 6], device='cuda:0')

In [ ]:
model(X,q)[6]

torch.Size([128, 1, 128])
torch.Size([128, 1, 7, 128])
torch.Size([128, 7, 128])
torch.Size([128, 1, 1, 128])
torch.Size([128, 128])
torch.Size([128, 10])


tensor([ 0.0326,  0.1267,  0.0637,  0.0893,  0.0409, -0.1043,  0.0812, -0.0475,
        -0.0947,  0.0971], device='cuda:0', grad_fn=<SelectBackward0>)

In [ ]:
cla[0]

tensor([6, 7, 1, 0, 6, 8, 9, 0, 0, 4, 5, 0, 5, 3, 1, 8], device='cuda:0')

In [ ]:
torch.argmax(rho.squeeze(-1), dim=-1)

tensor([ 8,  3, 14,  6, 11, 10, 15,  4,  8, 10, 12,  1,  9,  0, 11, 13,  2,  9,
         6, 11,  9, 11,  2, 13, 15,  0,  9,  4,  1, 11, 10,  3,  4,  7,  4, 10,
        10, 13,  2,  8,  6,  4,  0,  9,  4, 12,  4,  2,  8,  0, 15,  3,  0,  1,
        15,  5,  9, 11,  1,  3,  2,  0,  3,  7])

In [ ]:
cla.take(torch.argmax(rho.squeeze(-1), dim=-1))

tensor([5, 2, 4, 0, 5, 5, 7, 8, 5, 5, 8, 5, 5, 2, 5, 6, 5, 5, 0, 5, 5, 5, 5, 6,
        7, 2, 5, 8, 5, 5, 5, 2, 8, 6, 8, 5, 5, 6, 5, 5, 0, 8, 2, 5, 8, 8, 8, 5,
        5, 2, 7, 2, 2, 5, 7, 4, 5, 5, 5, 2, 5, 2, 2, 6])

In [ ]:
outputs = model(batch, q)

AttributeError: module 'torch.nn' has no attribute 'MultiHeadDotProductAttention'

In [ ]:
outputs = model(**batch)
next_word_logits = outputs.logits[:, num_prompts : -1, :]
loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))

TypeError: Model(
  (fc1): Linear(in_features=128, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=128, bias=True)
  (fc6): Linear(in_features=10, out_features=10, bias=True)
) argument after ** must be a mapping, not Tensor

In [ ]:
while loss.item() > 0.1:
  model.zero_grad()
  opt.zero_grad()
  outputs = model(**batch)
  next_word_logits = outputs.logits[:, num_prompts : -1, :]
  true_next_tokens = batch['input_ids'][:, num_prompts + 1:]
  loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))
  print("loss is {}".format(loss))
  loss.backward()
  opt.step()


NameError: name 'loss' is not defined